# The Movie DataBase (TMDB)

In [1]:
import pandas as pd
import numpy as np
import re
import json
from IPython.display import Markdown, display

## Data Ingestion

In [2]:
credits = pd.read_csv("data/tmdb_5000_credits.csv")
display(credits.head(), credits.shape)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


(4803, 4)

In [3]:
movies = pd.read_csv("data/tmdb_5000_movies.csv")
display(movies.head(3), movies.shape)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


(4803, 20)

## 1 - For each movie, compute the number of cast members

In [4]:
def n_cast(x): # per calcolare il numero di cast member
    x = json.loads(x)
    if (len(x) != 0):
        df = pd.DataFrame(x)['id'].unique() # vediamo se ci sono membri del cast ripetuti
        return len(df)
    else:
        return len(x)
credits['cast_members'] = credits['cast'].apply(n_cast) # creiamo nuova colonna con all'interno il numero di cast member
credits[['movie_id','cast_members']].head()

,movie_id,cast_members
0,19995,83
1,285,33
2,206647,83
3,49026,158
4,49529,27


In [5]:
app = pd.DataFrame(credits['cast'].apply(json.loads)[1]) # prendiamo l'elem di posizione 1 dove abbiamo notato che eliminando i valori ripetuti, si ha un numero di elementi del cast minore. Es.Keith Richards 'id'-1430
display("Es. movie_id: 285 abbiamo elementi del cast ripetuti", app[app['id']==1430], 
        "Primi 5 casi in cui cast_members = 0", credits[credits['cast_members']==0].head())

'Es. movie_id: 285 abbiamo elementi del cast ripetuti'

,cast_id,character,credit_id,gender,id,name,order
12,37,Captain Teague Sparrow,52fe4232c3a36847f800b5b3,2,1430,Keith Richards,12
31,65,Captain Teague,58bc2a37c3a368663003740b,2,1430,Keith Richards,31


'Primi 5 casi in cui cast_members = 0'

,movie_id,title,cast,crew,cast_members
2601,17644,Barney's Great Adventure,[],"[{""credit_id"": ""52fe473b9251416c750921ff"", ""de...",0
3670,447027,Running Forever,[],[],0
3992,346081,Sardaarji,[],"[{""credit_id"": ""558ab3f4925141076f0001d7"", ""de...",0
4009,126509,2016: Obama's America,[],"[{""credit_id"": ""52fe4ae1c3a368484e16bcd7"", ""de...",0
4068,371085,Sharkskin,[],[],0


## 2 - How many movies do not have a homepage?
Ovviamente si suppone che i film che non hanno una homepage siano impostati come nulli.

In [6]:
noHome = len(movies[movies['homepage'].isnull()]) # vediamo se null e prendiamo solo quelli, ovviamente usiamo len() per vedere quanti ne sono
print("Il numero di film che non hanno una homepage sono:", noHome)

Il numero di film che non hanno una homepage sono: 3091


## 3 - For each year, how many movies do not have a homepage?

In [7]:
movies['year'] = pd.to_datetime(movies['release_date']).dt.year # estraggo l'anno da release_date

# movies[movies['year'].isnull()] # vediamo che c'è un anno NaN questo perchè non è indicata la release_date
# movies.loc[4553]

m_noHome = movies[movies['homepage'].isnull()].groupby('year').size().reset_index().rename(columns = {0:'n_noHomepage'}) # prendiamo solo i film che non hanno una homepage, raggruppiamo per anno e calcoliamo quanti film sono presenti
m_noHome.year = m_noHome.year.astype(int) # diamo valori interi agli anni (il valore NaN precedente non permetteva ciò)
m_noHome.head()

,year,n_noHomepage
0,1916,1
1,1925,1
2,1927,1
3,1929,2
4,1930,1


## 4 - Extract the domain of each homepage

In [8]:
def multi_home(url): # per vedere se sono presenti più homepage
    url = re.split(' ', url)
    if (len(url) == 1):
        return False
    else:
        return True
multi = movies.homepage.dropna().apply(multi_home) # leviamo valori nulli altrimenti non sarebbe possibile applicare la funzione
movies.homepage.dropna()[multi]

3730    http://www.cargoderfilm.ch http://cargothemovi...
Name: homepage, dtype: object

In [9]:
movies[['homepage']][3725:3735] # notiamo che ci sono valori null e possono essere presenti più homepage (doppia quadra per vedere come dataframe)

,homepage
3725,NaN
3726,NaN
3727,http://www.filmensnabbacash.se/
3728,http://www.whaleriderthemovie.com/
3729,http://www.paathefilm.com/
3730,http://www.cargoderfilm.ch http://cargothemovi...
3731,http://tv.disney.go.com/disneychannel/original...
3732,NaN
3733,NaN
3734,http://www.miramax.com/movie/the-crying-game/


In [10]:
def domain(url):
    x = []
    if (pd.isnull(url)):
        return np.nan
    else:
        home = url.split(' ') # nel caso sia presente più di una homepage, vedere caso 3730: Cargo
        for i in range(len(home)):
            dom = re.findall(r'[\w\-+.]+', home[i])  # in dom[0] è presente http..., in dom[1] il dominio effettivo
            x.append(dom[1])
        return str(x)[1:-1] # trasformo in stringa levo parentesi quadre e apici
    
movies['domain'] = movies['homepage'].apply(domain)
movies[['title','homepage','domain']][3725:3735].dropna() # toglie solo quelli con homepage e quindi domain a NaN, i titoli sono presenti in ogni riga

,title,homepage,domain
3727,Easy Money,http://www.filmensnabbacash.se/,'www.filmensnabbacash.se'
3728,Whale Rider,http://www.whaleriderthemovie.com/,'www.whaleriderthemovie.com'
3729,Paa,http://www.paathefilm.com/,'www.paathefilm.com'
3730,Cargo,http://www.cargoderfilm.ch http://cargothemovi...,"'www.cargoderfilm.ch', 'cargothemovie.com'"
3731,High School Musical,http://tv.disney.go.com/disneychannel/original...,'tv.disney.go.com'
3734,The Crying Game,http://www.miramax.com/movie/the-crying-game/,'www.miramax.com'


## 5 - Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary). There is no need to use SQL for this point